<a href="https://colab.research.google.com/github/mpnsk/ivy_seminar/blob/main/ivy_complete_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ivy
!pip install dm-haiku

In [12]:
import ivy

In [28]:

my_backend = "torch"
# my_backend = "tensorflow"
# my_backend = "jax"

doCompile = False

if my_backend == "jax":
  import jax
  jax.config.update('jax_enable_x64', True)

ivy.set_backend(my_backend)

class Net(ivy.Module):
    def __init__(self):
        self.conv1 = ivy.Conv2D(1, 32, [3, 3], 1, "VALID")
        self.conv2 = ivy.Conv2D(32, 64, [3, 3], 1, "VALID")
        self.conv1_drop = ivy.Dropout(0.25)
        self.conv2_drop = ivy.Dropout(0.5)
        self.fc1 = ivy.Linear(9216, 128)
        self.fc2 = ivy.Linear(128, 10)
        super().__init__()

    def _forward(self, x):
        x = self.conv1(x)
        x = ivy.relu(x)
        x = self.conv2(x)
        x = ivy.relu(x)
        x = ivy.max_pool2d(x, 2, 2, 'VALID')
        x = self.conv1_drop(x)
        x = ivy.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = ivy.relu(x)
        x = self.conv2_drop(x)
        x = self.fc2(x)
        output = ivy.softmax(x)
        return output


model = Net()
arg = ivy.random_normal(shape=(1,28,28,1))
if doCompile:
  print("starting compile")
  model = ivy.compile(model, to=my_backend, args=(arg,))
print("done")

done


In [29]:
print('loading mnist')
from keras.datasets import mnist
(train_X_np, train_y_np), (test_X, test_y) = mnist.load_data()

limit = 6400
batch_size = 64
num_batches = limit // batch_size

train_X_np = train_X_np[:limit, :, :]
train_y_np = train_y_np[:limit]

print(f'{train_X_np.shape=}')

# my_x = ivy.empty((1, 28, 28, 1))
# my_x[:, :, :, 0] = ivy.array(train_X[0])

print('defined model')


def loss_fn(v, x, y):
    y_pred = model(x)
    entropy = ivy.sparse_cross_entropy(y, y_pred)
    aggregated_entropy = entropy.mean()
    return aggregated_entropy


# jax.numpy.set_printoptions(threshold=1000000)

print('starting to batch')
# turn training data into batches
## convert to ivy
train_X_ivy = ivy.array(train_X_np)
train_y_ivy = ivy.array(train_y_np)

# normalize grey scale to 0.0-1.0
train_X_ivy = train_X_ivy / 255

train_X_batches = [train_X_ivy[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
train_y_batches = [train_y_ivy[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
print('done')

loading mnist
train_X_np.shape=(6400, 28, 28)
defined model
starting to batch
done


In [ ]:
for idx in range(num_batches):
    x = train_X_batches[idx]
    x = ivy.expand_dims(x, axis=-1)
    y = train_y_batches[idx]
    loss, grads = ivy.execute_with_gradients(lambda v: loss_fn(v, x, y), model.v)
    print(f'{idx=}, {loss=}')
    if grads.all().cont_all_false():
        print("ERROR!")
        print(f'{grads.conv1.w=}')
        print(f'{grads.conv1.b=}')
        print(f'{grads.conv2.w=}')
        print(f'{grads.conv2.b=}')
        print(f'{grads.fc1.w=}')
        print(f'{grads.fc1.b=}')
        print(f'{grads.fc2.w=}')
        print(f'{grads.fc2.b=}')
        break
    model.v = model.v - grads * 0.005

random_value = 127

x = train_X_ivy[random_value]
# expand hw to nhwc
x = ivy.expand_dims(x, axis=0)
x = ivy.expand_dims(x, axis=-1)
prediction = model(x)
print(f'{prediction=}')
print(f'{ivy.argmax(prediction)=}')
print(f'{train_y_ivy[random_value]=}')

idx=0, loss=ivy.array(2.7533603, dev=gpu:0)
idx=1, loss=ivy.array(2.436123, dev=gpu:0)
idx=2, loss=ivy.array(2.6664448, dev=gpu:0)
idx=3, loss=ivy.array(2.2585895, dev=gpu:0)
idx=4, loss=ivy.array(2.145646, dev=gpu:0)
idx=5, loss=ivy.array(2.0307014, dev=gpu:0)
idx=6, loss=ivy.array(2.1277814, dev=gpu:0)
idx=7, loss=ivy.array(2.0769477, dev=gpu:0)
idx=8, loss=ivy.array(1.9394872, dev=gpu:0)
idx=9, loss=ivy.array(1.9906547, dev=gpu:0)
idx=10, loss=ivy.array(1.9769748, dev=gpu:0)
idx=11, loss=ivy.array(2.0795205, dev=gpu:0)
idx=12, loss=ivy.array(1.7615812, dev=gpu:0)
idx=13, loss=ivy.array(1.9672122, dev=gpu:0)
idx=14, loss=ivy.array(1.9623533, dev=gpu:0)
idx=15, loss=ivy.array(1.6534996, dev=gpu:0)
idx=16, loss=ivy.array(2.0127206, dev=gpu:0)
idx=17, loss=ivy.array(1.9439561, dev=gpu:0)
idx=18, loss=ivy.array(1.7639469, dev=gpu:0)
idx=19, loss=ivy.array(1.7422255, dev=gpu:0)
idx=20, loss=ivy.array(1.8011327, dev=gpu:0)
idx=21, loss=ivy.array(1.721169, dev=gpu:0)
idx=22, loss=ivy.array(

Exception ignored in: <function _xla_gc_callback at 0x7bba84c09360>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


idx=26, loss=ivy.array(1.4897201, dev=gpu:0)
idx=27, loss=ivy.array(1.5122917, dev=gpu:0)
idx=28, loss=ivy.array(1.5331728, dev=gpu:0)
idx=29, loss=ivy.array(1.3514293, dev=gpu:0)
